# **미니프로젝트 1대1 문의 내용 유형 분류기**
# Text classification

### 문제 정의
> 1:1 문의 내용 분류 문제<br>
> 1. 문의 내용 분석
> 2. 문의 내용 분류 모델 성능 평가
### 학습 데이터
> * 1:1 문의 내용 데이터 : train.csv

### 변수 소개
> * text : 문의 내용
> * label : 문의 유형

## 1. 개발 환경 설정

### 라이브러리 import

In [44]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import pandas as pd
import matplotlib.font_manager as fm
from tqdm import tqdm
import os

## 2. Using pre-trained model

### 2-1 KoBert 구현
### MultiClassification

[참고 blog : https://velog.io/@seolini43/KOBERT로-다중-분류-모델-만들기-파이썬Colab]

* Transformer 안에 sentencepicece 모듈이 버전업 하면서 문제가 발생
* HuggingFace API 버전으로 수정

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#kobert
from transformers import BertModel, DistilBertModel

bert_model = BertModel.from_pretrained('monologg/kobert') # hugging face ver. kobert parameter download

In [4]:
from tokenization_kobert import KoBertTokenizer

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert') # monologg/distilkobert도 동일

print(tokenizer.tokenize("[CLS] 한국어 모델을 공유합니다. [SEP]"))
print('-'*50)
print(tokenizer.convert_tokens_to_ids(['[CLS]', '▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.', '[SEP]']))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


['[CLS]', '▁한국', '어', '▁모델', '을', '▁공유', '합니다', '.', '[SEP]']
--------------------------------------------------
[2, 4958, 6855, 2046, 7088, 1050, 7843, 54, 3]


In [5]:
# pip install kobert-transformers

## install library

In [6]:
#GPU 사용
import torch

device = torch.device("cuda:0")
print(device)

cuda:0


In [60]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import tqdm
import pickle

from kobert_transformers import get_kobert_model, get_distilkobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

#BERT 모델, Vocabulary 불러오기
bertmodel = get_kobert_model()

In [8]:
# label_dict = {
#     '코드1': 0,
#     '코드2': 0,
#     '웹': 1,
#     '이론': 2,
#     '시스템 운영': 3,
#     '원격': 4
# }

# df_test['label'] = df_test['label'].map(label_dict)

In [9]:
import pandas as pd

df_test=pd.read_csv("data/data_add_qna_pos_nouns.csv", index_col = 0)
df_test.tail()

,text,label,spaced_sent,remove_special_sent,en_stop_nouns,en_stop_pos,en_code_nouns,en_code_pos,en_stop_morphs
3859,stop_words = set(stopwords.words('english')) ...,2,stop_words = set(stopw ords.words('english')) ...,stop_words = setstopw ordswordsenglish 이 부분이 어...,stop words setstopw 부분 의 미 알 수,"[('stop', 'NN'), ('words', 'NNS'), ('setstopw'...",부분 의 미 알 수,"[('이', 'IC'), ('부분', 'NNG'), ('이', 'JKS'), ('어...","['stop', 'words', 'setstopw', 'ordswordsenglis..."
3860,Tokenizer로 전처리를 한 이유가 무엇인가요? 실습 중 Tokenizer를 사...,2,Tokenizer로 전처리를 한 이유가 무엇인가요? 실습 중 Tokeniz er를 ...,Tokenizer로 전처리를 한 이유가 무엇인가요? 실습 중 Tokeniz er를 ...,tokenizer tokeniz er tokeniz er train inputs t...,"[('tokenizer', 'NN'), ('tokeniz', 'NN'), ('er'...",tokenizer train train 전 처리 이유 무엇 실습 중 면수 이유 위 ...,"[('tokenizer', 'NN'), ('train', 'NN'), ('train...","['tokenizer', '처리', '이유', '무엇', 'ㄴ가', '실습', 't..."
3861,stanford.edu에서 data_set을 가져올 때 저장 경로는 어떻게 되는지...,2,stanford. edu에서 data _set을 가져올 때 저장 경로는 어떻게 되는...,stanford edu에서 data _set을 가져올 때 저장 경로는 어떻게 되는지...,stanford edu data set file 때 저장 경로 실습 때 로컬 어디,"[('stanford', 'NN'), ('edu', 'NN'), ('data', '...",set 때 저장 경로 실습 때 로컬 어디,"[('set', 'NN'), ('에서', 'JKB'), ('을', 'JKO'), (...","['stanford', 'edu', '에서', 'data', 'set', '가져오'..."
3862,거리에 대해서 얘기해 주시는데 이 거리가 언어지능과 어떤 부분에서 관련되는지 궁금합...,2,거리에 대해서 얘기해 주시는 데 이 거리가 언어지능과 어떤 부분에서 관련되는지 궁금...,거리에 대해서 얘기해 주시는 데 이 거리가 언어지능과 어떤 부분에서 관련되는지 궁금...,거리 데 거리 언어 지능 부분 언어 벡터 벡터 거리 유사 차이 거 리 거,"[('거리', 'NNG'), ('에', 'JKB'), ('대하', 'VV'), ('...",거리 데 거리 언어 지능 부분 언어 벡터 벡터 거리 유사 차이 거 리 거,"[('거리', 'NNG'), ('에', 'JKB'), ('대하', 'VV'), ('...","['거리', '대하', '아서', '얘기하', '거리', '언어', '지능', '어..."
3863,cosine similarity에 대한 질문입니다. 강의 자료에서는 cosine m...,2,cos ine similarity에 대한 질문입니다. 강의 자료에서는 cos ine...,cos ine similarity에 대한 질문입니다 강의 자료에서는 cos ine ...,cos ine similarity cos ine measure f rom sklea...,"[('cos', 'NN'), ('ine', 'NN'), ('similarity', ...",import 질문 강의 자료 값 이상 이하 값 수록 것 실습 코드 값 유사 것 제 ...,"[('import', 'NN'), ('에', 'IC'), ('대하', 'VV'), ...","['cos', 'ine', 'similarity', '대하', '질문', 'ㅂ니다'..."


In [10]:
test = pd.read_csv("data/test.csv", index_col = 0)

test.head()

,text
id,
0,팀즈는 노트북으로 접속하고 강의는 데스크톱 이용하려고 하는데 문제는 없을까요? 이...
1,"셀프테스트에서 받은 점수가 해당 교육을 이수하고, 취업 연계등을 하는데 있어서 영향..."
2,a= int(input('정수 A의 값을 입력하시오.:'))\nb= int(inpu...
3,"def max4(a,b,c,d):\n maximum = a\n if b&..."
4,i**=2 가 i **=2 와 왜 같은지가 이해가 잘 안됩니다!!


In [11]:
data_list = []
for q, label in zip(df_test['remove_special_sent'], df_test['label'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [12]:
print(data_list[0])
print(data_list[1000])
print(data_list[2000])
print(data_list[-1])

['self convs1 = nn ModuleListn nConv 2d1 Co K 100 for K in Ks 1 여기서 커널사이즈 K 100 은 한 단어의 최대 길이가 100이 넘지 않는다는 가정으로 그냥 100으로 선언한 것인지 궁금합니다', '0']
['x값이 너무 많아서 겹쳐서 검은색으로 보이는데요 아마 온도값이 01 단위로 써 있어서 그렇지 않을까 추측합니다 혹시 온도 단위를 05나 1 단위로 보고 싶으면 어떤 코드를 추가해야 할까요? bins를 넣으면 x값 단위는 그대로이고 막대 그래프 개수만 줄어서 효과가 없는 것 같습니다', '0']
['수업 끝난 후 과 제 제출을 위해 접속을 했는데 계속 접속이 안 됩니다', '3']
['cos ine similarity에 대한 질문입니다 강의 자료에서는 cos ine measure 값이 0 이상 2 이하의 값이 나오며 작을 수록 유사한 것이라고 이해했습니다 실습 코드에서 사용한 f rom sklearnmet ricspairwise import cosine_simil arity는 값이 클수록 유사 한 것 같아 제가 제대로 이해한 것이 맞나요? 코 드에서 사용 한 cos ine_simil arity와 cos ine measure는 다른 것인가요?', '2']


In [32]:
texts = [item[0] for item in data_list]
labels = [item[1] for item in data_list]

In [33]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [34]:
print(len(train_texts))
print(len(test_texts))

3091
773


In [59]:
# Setting parameters
max_len = 128
batch_size = 16
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate = 2e-5

In [80]:
import torch
from torch.utils.data import Dataset

def bert_tokenizer(sent, MAX_LEN, tokenizer):
    
    encoded_dict=tokenizer.encode_plus(
    text = sent, 
    add_special_tokens=True, 
    max_length=MAX_LEN, 
    pad_to_max_length=True, 
    return_attention_mask=True,
    truncation = True)
    
    input_id=encoded_dict['input_ids']
    attention_mask=encoded_dict['attention_mask']
    #token_type_id = encoded_dict['token_type_ids']
    token_type_id = 0
    
    return input_id, attention_mask, token_type_id

def preprocessing_train():
    
    pt = 'monologg/kobert'
    
    tokenizer = KoBertTokenizer.from_pretrained(pt,  cache_dir='bert_ckpt', do_lower_case=False)
    print('load kobert')
    
    MAX_LEN = 128
    data = pd.read_csv('./data/data_add_qna_pos_nouns.csv')
    valid = int(len(data) * 0.8)

    train=data.loc[:valid,['text','label']]
    valid=data.loc[valid:,['text','label']]

    input_ids =[]
    attention_masks =[]
    token_type_ids =[]
    train_data_labels = []

    for train_sent, train_label in tqdm.tqdm(zip(train['text'], train['label'])):
        try:
            input_id, attention_mask,_ = bert_tokenizer(train_sent, MAX_LEN=MAX_LEN, tokenizer=tokenizer)

            input_ids.append(input_id)
            attention_masks.append(attention_mask)
            token_type_ids.append(0)
            #########################################
            train_data_labels.append(train_label)

        except Exception as e:
            print(e)
            pass

    train_input_ids=np.array(input_ids, dtype=int)
    train_attention_masks=np.array(attention_masks, dtype=int)
    train_token_type_ids=np.array(token_type_ids, dtype=int)
    ###########################################################
    train_inputs=(train_input_ids, train_attention_masks, train_token_type_ids)
    train_labels=np.asarray(train_data_labels, dtype=np.int32)

    # save
    train_data = {}

    train_data['input_ids'] = train_input_ids
    train_data['attention_mask'] = train_attention_masks
    train_data['token_type_ids'] = train_token_type_ids
    train_data['targets'] = np.asarray(train_data_labels, dtype=np.int32)
    
    os.makedirs(f'./data/{pt}/', exist_ok=True)
    with open(f'./data/{pt}/train_data_{MAX_LEN}.pickle', 'wb') as f:
        pickle.dump(train_data, f, pickle.HIGHEST_PROTOCOL)

    # valid

    input_ids =[]
    attention_masks =[]
    token_type_ids =[]
    valid_data_labels = []

    for valid_sent, valid_label in tqdm.tqdm(zip(valid['text'], valid['label'])):
        try:
            input_id, attention_mask,_ = bert_tokenizer(valid_sent, MAX_LEN=MAX_LEN, tokenizer=tokenizer)

            input_ids.append(input_id)
            attention_masks.append(attention_mask)
            token_type_ids.append(0)
            #########################################
            train_data_labels.append(train_label)

        except Exception as e:
            print(e)
            pass

    valid_input_ids=np.array(input_ids, dtype=int)
    valid_attention_masks=np.array(attention_masks, dtype=int)
    valid_token_type_ids=np.array(token_type_ids, dtype=int)
    ###########################################################
    valid_inputs=(valid_input_ids, valid_attention_masks, valid_token_type_ids)
    valid_labels=np.asarray(valid_data_labels, dtype=np.int32)

    # save
    valid_data = {}

    valid_data['input_ids'] = valid_input_ids
    valid_data['attention_mask'] = valid_attention_masks
    valid_data['token_type_ids'] = valid_token_type_ids
    valid_data['targets'] = np.asarray(valid_data_labels, dtype=np.int32)
    
    os.makedirs(f'./data/{pt}/', exist_ok=True)
    with open(f'./data/{pt}/valid_data_{MAX_LEN}.pickle', 'wb') as f:
        pickle.dump(valid_data, f, pickle.HIGHEST_PROTOCOL)

def preprocessing_test():
    
    pt = 'monologg/kobert'
    if 'kobert' in pt:
        tokenizer = KoBertTokenizer.from_pretrained(pt,  cache_dir='bert_ckpt', do_lower_case=False)
        print('load kobert')

    MAX_LEN = 128
    
    test = pd.read_csv('./data/test.csv')
    test=test[['text']]
    
    input_ids =[]
    attention_masks =[]
    token_type_ids =[]

    for test_sent in tqdm.tqdm(test['text']):
        try:
            input_id, attention_mask,_ = bert_tokenizer(test_sent, MAX_LEN=MAX_LEN, tokenizer=tokenizer)

            input_ids.append(input_id)
            attention_masks.append(attention_mask)
            token_type_ids.append(0)
            #########################################

        except Exception as e:
            print(e)
            pass

    test_input_ids=np.array(input_ids, dtype=int)
    test_attention_masks=np.array(attention_masks, dtype=int)
    test_token_type_ids=np.array(token_type_ids, dtype=int)
    ###########################################################
    test_inputs=(test_input_ids, test_attention_masks, test_token_type_ids)


    # save
    test_data = {}

    test_data['input_ids'] = test_input_ids
    test_data['attention_mask'] = test_attention_masks
    test_data['token_type_ids'] = test_token_type_ids
    
    os.makedirs(f'./data/{pt}/', exist_ok=True)
    with open(f'./data/{pt}/test_data_{MAX_LEN}.pickle', 'wb') as f:
        pickle.dump(test_data, f, pickle.HIGHEST_PROTOCOL)


In [81]:
preprocessing_train()
preprocessing_test()
        
print('모델 전처리 완료')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


load kobert


3092it [00:00, 4304.67it/s]
773it [00:00, 4523.26it/s]
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


load kobert


100%|██████████| 929/929 [00:00<00:00, 4506.70it/s]

모델 전처리 완료


In [82]:
class KobertDataSet(Dataset):
    
    def __init__(self, data, test=False):
        
        self.data = data
        self.test = test
        
    def __len__(self):
        
        return self.data['input_ids'].shape[0]
    
    def __getitem__(self,idx):
        
        ids = torch.tensor(self.data['input_ids'][idx], dtype=torch.long)
        mask = torch.tensor(self.data['attention_mask'][idx], dtype=torch.long)
        token_type_ids = torch.tensor(self.data['token_type_ids'][idx], dtype=torch.long)
         
            
        if self.test:
            return {
                'ids': ids,
                'mask': mask,
                'token_type_ids': token_type_ids
            }
        
        else:
            target = torch.tensor(self.data['targets'][idx],dtype=torch.long)

            return {
                    'ids': ids,
                    'mask': mask,
                    'token_type_ids': token_type_ids,
                    'targets': target
                }

In [83]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [84]:
with open(f'./data/monologg/kobert/train_data_128.pickle', 'rb') as f:
    train_data = pickle.load(f)
with open(f'./data/monologg/kobert/valid_data_128.pickle', 'rb') as f:
    valid_data = pickle.load(f)

train_dataset = KobertDataSet(data = train_data)
valid_dataset = KobertDataSet(data = valid_data)

trainloader = DataLoader(dataset=train_dataset, batch_size=batch_size,
                                 num_workers=8, shuffle=True, pin_memory=True)
validloader = DataLoader(dataset=valid_dataset, batch_size=batch_size, 
                                 num_workers=8, shuffle=False, pin_memory=True)

In [85]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(trainloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

trainloader

In [86]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0

    model.train()
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm.tqdm(trainloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm.tqdm(validloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/194 [00:00<?, ?it/s]

In [ ]:
import os
os.chdir('/content/drive/MyDrive/models/')
os.getcwd()

path = '/content/drive/MyDrive/models/'
torch.save(model, path + '5_Tags_model.pt')  # 전체 모델 저장

torch.save(model.state_dict(), '5_Tags_model_state_dict.pt')  # 모델 객체의 state_dict 저장

torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, '5_Tags_model_all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

In [ ]:
pwd

'/content/drive/MyDrive/models'

In [ ]:
import os
os.chdir('/content/drive/MyDrive/models/')

model1 = torch.load('5_Tags_model.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model1.load_state_dict(torch.load('5_Tags_model_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

checkpoint = torch.load('5_Tags_model_all.tar')   # dict 불러오기
model1.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/drive/MyDrive/models/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
label_dict

{'코드1': 0, '코드2': 0, '웹': 1, '이론': 2, '시스템 운영': 3, '원격': 4}

In [ ]:
test_data=pd.read_csv("/content/drive/MyDrive/aivle_school_mini_project_4/2023.04.03_미니프로젝트4차_실습자료/test_prepro_BERT.csv")
test_data.head()

,id,text
0,0,팀즈는 노트북으로 접속하고 강의는 데스크톱 이용하려고 하는데 문제는 없을까요? 이 ...
1,1,셀프테스트에서 받은 점수가 해당 교육을 이수하고 취업 연계 등을 하는데 있어서 영향...
2,2,a= intinput정수 A의 값을 입력하시오: b= intinput정수 B의 값...
3,3,def max4a b c d: maxim um = a if bgtmaxim um: ...
4,4,i= 2 가 i = 2 와 왜 같은 지가 이해가 잘 안 됩니다!!


In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("코드1,2")
            elif np.argmax(logits) == 1:
                test_eval.append("웹")
            elif np.argmax(logits) == 2:
                test_eval.append("이론")
            elif np.argmax(logits) == 3:
                test_eval.append("시스템 운영")
            elif np.argmax(logits) == 4:
                test_eval.append("원격격")

        print('input : ', predict_sentence)
        print("--------------------------------")
        print(">> 해당 문의는는 " + test_eval[0] + " 입니다.")
    return np.argmax(logits)

using cached model. /content/KoBERT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
answer = []

for text in test_data['text']:
  sentence = text
  pred_y = predict(sentence)
  answer.append(pred_y)
  print("")

print(answer)

input :  팀즈는 노트북으로 접속하고 강의는 데스크톱 이용하려고 하는데 문제는 없을까요? 이 경우에는 다운 받아야 되는 파일이 있을까요? 노트북 트래픽이 많아지면 소리 부분 끊기거나 전체 화면을 왔다 갔다 하니까 놓치는 부분이 있어서 질문드립니다
--------------------------------
>> 해당 문의는는 시스템 운영 입니다.

input :  셀프테스트에서 받은 점수가 해당 교육을 이수하고 취업 연계 등을 하는데 있어서 영향을 미치는 요소인가요?
--------------------------------
>> 해당 문의는는 시스템 운영 입니다.

input :  a= intinput정수 A의 값을 입력하시오:  b= intinput정수 B의 값을 입력 하시오:  c= intinput정수 C의 값을 입력하시오:  d= intinput정수 D의 값을 입력하시오:  maxim um = a if bgt  maxim um : b= maxim um if cgt maximu m: c= maxim um if dgt  maxim um: d= maxim um printf최대값은 maxim um 입니다 왜 최초에 설정한 a값만 나오는 지 알 수 있을까요?
--------------------------------
>> 해당 문의는는 코드1,2 입니다.

input :  def max4a b c d: maxim um = a if bgtmaxim um: maxim um =b if cgtmaxim um: maxim um=c if dgtmaxim um: maxim um=d retu rn maximum printf max44 32 1 =  max44 3 2 1  printf max451 2 3 = max45 1 2 3  printf max4254 7 = max425 4 7  printf max4213 1 = max421 3 1  printf max4111 1 = max411 1 1  printf max4 7 4 5 9 = max47 45 9 
-----------

In [ ]:
submit = pd.read_csv("/content/drive/MyDrive/aivle_school_mini_project_4/2023.04.03_미니프로젝트4차_실습자료/random_submission.csv")

submit

,id,label
0,0,2
1,1,0
2,2,3
3,3,3
4,4,4
...,...,...
924,924,4
925,925,3
926,926,3
927,927,2


In [ ]:
submit['label'] = answer

In [ ]:
submit.to_csv("/content/drive/MyDrive/aivle_school_mini_project_4/2023.04.03_미니프로젝트4차_실습자료/data/submit_plz_final.csv", index=False)

### KoElectraForSequence

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 94.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm

In [ ]:
# GPU 사용
device = torch.device("cuda")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/aivle_school_mini_project_4/2023.04.03_미니프로젝트4차_실습자료/train.csv')

df.head()

,text,label
0,"self.convs1 = nn.ModuleList([nn.Conv2d(1, Co, ...",코드2
1,현재 이미지를 여러개 업로드 하기 위해 자바스크립트로 동적으로 폼 여러개 생성하는데...,웹
2,glob.glob(PATH) 를 사용할 때 질문입니다.\n\nPATH에 [ ] 가 ...,코드2
3,"tmpp = tmp.groupby(by = 'Addr1', as_index=Fals...",코드2
4,filename = TEST_IMAGE + str(round(frame_sec)) ...,코드2


In [ ]:
from sklearn.model_selection import train_test_split
x = df.loc[:, 'text']
y = df.loc[:, 'label']

train_df, test_df, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=2023)

In [ ]:
train_copy = pd.concat([train_df, train_y], axis=1)
test_copy = pd.concat([test_df, test_y], axis=1)

train_copy.to_csv("train_df.csv", index=False)
test_copy.to_csv("test_df.csv", index=False)

In [ ]:
label_dict = {
    '코드1': 0,
    '코드2': 0,
    '웹': 1,
    '이론': 2,
    '시스템 운영': 3,
    '원격': 4}

class NSMCDataset(Dataset):

  def __init__(self, csv_file):
    # 일부 값중에 NaN이 있음...
    self.dataset = pd.read_csv(csv_file).dropna(axis=0)
    self.dataset['label'] = self.dataset['label'].map(label_dict)
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")

    print(self.dataset.describe())

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 0:2].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [ ]:
train_dataset = NSMCDataset('train_df.csv')
test_dataset = NSMCDataset("test_df.csv")

             label
count  2964.000000
mean      1.152834
std       1.202114
min       0.000000
25%       0.000000
50%       1.000000
75%       2.000000
max       4.000000
            label
count  742.000000
mean     1.150943
std      1.212338
min      0.000000
25%      0.000000
50%      1.000000
75%      2.000000
max      4.000000


In [ ]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator").to(device)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

In [ ]:
# 모델 레이어 보기
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
epochs = 5
batch_size = 16

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
losses = []
accuracies = []

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (torch.max(y_pred, 1)[1] == y_batch).sum().item() # 정확도 계산 방법 변경
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)

  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

  0%|          | 0/186 [00:00<?, ?it/s]

RuntimeError: ignored